In [141]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [142]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/ml_plus/data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [155]:
from surprise import BaselineOnly, Dataset, Reader, SVD
from surprise.model_selection import cross_validate
import pandas as pd
import scipy
import numpy as np

In [156]:
ratings = pd.read_csv('my_rating.csv')

In [157]:
movies = pd.read_csv('영화 & TV 프로그램 .csv', index_col= 0)

In [158]:
movies = movies.drop(['content','imgUrl','createdYear','type'],axis=1)

In [159]:
movies = movies.drop('contentRating',axis=1)

In [160]:
movies.columns = ['movieId','title','genres']

In [161]:
ratings = ratings.rename(columns={'contentReviewUserId':'userId'})
ratings = ratings.rename(columns={'contentId':'movieId'})
ratings = ratings.rename(columns={'userRating':'rating'})

In [162]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [166]:
# SVD 알고리즘으로 모델 학습
algo = SVD()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# 모델을 전체 데이터로 학습시킴
trainset = data.build_full_trainset()
algo.fit(trainset)

# 특정 유저가 본 영화를 제외하고 가장 높은 예측값을 가진 영화를 추천
user_id = 5
user_movie_ids = ratings[ratings.userId == user_id]['movieId'].values
user_ratings = [(i, algo.predict(user_id, i).est) for i in movies.movieId if i not in user_movie_ids]
user_ratings = sorted(user_ratings, key=lambda x: x[1], reverse=True)[:10]
user_recommendations = movies[movies.movieId.isin([x[0] for x in user_ratings])]
user_recommendations['recommendation_score'] = [x[1] for x in user_ratings]

# 결과를 보기 좋게 출력
print("추천 영화 목록:")
for i, row in user_recommendations.iterrows():
    print("{}. {} [{}] ({:.2f})".format(i+1, row['title'], row['genres'], row['recommendation_score']))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1572  1.1274  1.2118  1.2059  1.1632  1.1731  0.0316  
MAE (testset)     1.0004  0.9740  1.0653  1.0324  0.9979  1.0140  0.0317  
Fit time          0.02    0.03    0.03    0.03    0.02    0.03    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


<ipython-input-166-956778bc5720>:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  user_ratings = [(i, algo.predict(user_id, i).est) for i in movies.movieId if i not in user_movie_ids]


추천 영화 목록:
1. 똑똑똑 [['Horror', 'Mystery', 'Thriller']] (3.00)
2. 블랙 팬서: 와칸다 포에버 [['Action', 'Adventure', 'Science Fiction']] (3.00)
3. 장화신은 고양이: 끝내주는 모험 [['Animation', 'Adventure', 'Comedy', 'Family']] (3.00)
4. Shark Side of the Moon [['Action', 'Science Fiction', 'Thriller', 'Horror']] (3.00)
5. 플레인 [['Action', 'Adventure', 'Thriller']] (3.00)
6. 리틀 딕시 [['Thriller', 'Action', 'Drama', 'Crime']] (3.00)
7. 납골당 [['Drama', 'Horror', 'Mystery']] (3.00)
8. 메간 [['Science Fiction', 'Horror', 'Comedy']] (3.00)
9. 밴디트 [['Crime', 'Drama', 'Thriller']] (3.00)
10. 아바타: 물의 길 [['Science Fiction', 'Adventure', 'Action']] (3.00)


<ipython-input-166-956778bc5720>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations['recommendation_score'] = [x[1] for x in user_ratings]


피클 파일 저장

In [167]:
# import pickle

# # 모델 객체를 피클 파일로 저장
# with open('recommendation_model.pkl', 'wb') as f:
#     pickle.dump(algo, f)

피클 파일 로드시 참고

In [151]:
# import pickle

# # 피클 파일에서 모델 객체를 로드
# with open('recommendation_model.pkl', 'rb') as f:
#     algo = pickle.load(f)

# # 추천 서비스를 사용하는 코드
# user_id = 5
# user_movie_ids = ratings[ratings.userId == user_id]['movieId'].values
# user_ratings = [(i, algo.predict(user_id, i).est) for i in movies.movieId if i not in user_movie_ids]
# user_ratings = sorted(user_ratings, key=lambda x: x[1], reverse=True)[:10]
# user_recommendations = movies[movies.movieId.isin([x[0] for x in user_ratings])]
# user_recommendations['recommendation_score'] = [x[1] for x in user_ratings]